## Setup

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Input, BatchNormalization, ReLU, Conv2DTranspose, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import GlobalAveragePooling2D, Reshape, Permute, multiply, Activation
from tensorflow.keras import Model
import numpy as np
import matplotlib.pyplot as plt
import os
from skimage.io import imread, imshow
from skimage.transform import resize
from skimage.measure import regionprops, label
from skimage.color import rgb2hsv, rgb2gray
import cv2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Preparing the Data




In [ ]:
batch_size = 10
img_height = 400
img_width = 400
rescale = 1./255
data_dir = '/DDS3/'


def preprocessor_mask(image):
   image = (255-image)
   image = image[:,:,0]
   image = np.reshape(image, (img_width, img_height, 1))
   return image

def preprocessor_weight(image):
   image = (255-image)
   image = image/255
   class_weights = tf.constant([1.0406, 50.2519])
   class_weights = class_weights/tf.reduce_sum(class_weights)
   sample_weights = tf.gather(class_weights, indices=tf.cast(image, tf.int32))
   return np.asarray(sample_weights)

seed = 909
image_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=rescale)
mask_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=rescale,
                                                               preprocessing_function=preprocessor_mask)
weight_datagen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocessor_weight)

image_generator_train =image_datagen.flow_from_directory(data_dir + 'TRAINING SET/IMAGE',
                                                    class_mode=None,
                                                    batch_size = batch_size,
                                                    target_size = (img_width, img_height),
                                                    seed = seed,
                                                    shuffle=False)
mask_generator_train = mask_datagen.flow_from_directory(data_dir + 'TRAINING SET/MASK',
                                                   class_mode=None,
                                                   batch_size = batch_size,
                                                   color_mode = 'grayscale',
                                                   target_size = (img_width, img_height),
                                                   seed = seed,
                                                   shuffle=False)

weight_generator_train = weight_datagen.flow_from_directory(data_dir + 'TRAINING SET/MASK',
                                                   class_mode=None,
                                                   batch_size = batch_size,
                                                   color_mode = 'grayscale',
                                                   target_size = (img_width, img_height),
                                                   seed = seed,
                                                   shuffle=False)

train_generator = zip(image_generator_train, mask_generator_train, weight_generator_train)

image_generator_valid =image_datagen.flow_from_directory(data_dir + 'VALIDATION SET/IMAGE',
                                                   class_mode=None,
                                                   batch_size = batch_size,
                                                   target_size = (img_width, img_height),
                                                   seed = seed,
                                                   shuffle=False)
mask_generator_valid = mask_datagen.flow_from_directory(data_dir + 'VALIDATION SET/MASK',
                                                   class_mode=None,
                                                   batch_size = batch_size,
                                                   target_size = (img_width, img_height),
                                                   seed = seed,
                                                   color_mode = 'grayscale',
                                                   shuffle=False)
valid_generator = zip(image_generator_valid, mask_generator_valid)

## Training

In [ ]:
num_classes = 2

inputs = tf.keras.layers.Input((400, 400, 3))

c1 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(inputs)
c1 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

c2 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
c2 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)

c3 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
c3 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)

c4 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
c4 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)

c5 = tf.keras.layers.Conv2D(1024, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
c5 = tf.keras.layers.Conv2D(1024, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)

u6 = tf.keras.layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(c5)
u6 = tf.keras.layers.concatenate([u6, c4])
u6 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
u6 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)


u7 = tf.keras.layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(u6)
u7 = tf.keras.layers.concatenate([u7, c3])
u7 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
u7 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)


u8 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(u7)
u8 = tf.keras.layers.concatenate([u8, c2])
u8 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
u8 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)

u9 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(u8)
u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
u9 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
u9 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)


outputs = tf.keras.layers.Conv2D(num_classes, (1, 1), activation='softmax')(u9)
model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
model.summary()

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                metrics=['accuracy'])

In [ ]:
STEP_SIZE_TRAIN=6000//batch_size
STEP_SIZE_VALID=1000//batch_size

model.fit(train_generator,
          steps_per_epoch=STEP_SIZE_TRAIN,
          validation_data=valid_generator,
          validation_steps=STEP_SIZE_VALID,
          epochs=30)

#### Save the Model

In [ ]:
model.save('model_segmentation_UNet.h5')

##Evaluating

In [ ]:
def loadImages(path, subfolder):
    '''Put files into lists and return them as one list with all images
     in the folder'''
    image_files = sorted([os.path.join(path, subfolder, file)
                          for file in os.listdir(path + "/" + subfolder)
                          if file.endswith('.bmp')])

    return image_files

In [ ]:
def create_mask(pred_mask):
  pred_mask = tf.argmax(pred_mask, axis=-1)
  pred_mask = pred_mask[..., tf.newaxis]
  return pred_mask

def segmentation_images(img_mosaico):
    image = img_mosaico/255
    m, n, c = image.shape
    image = np.reshape(image, (1, m, n, c))
    pred_mask = model(image, training=False)
    Y_new = np.asarray(create_mask(pred_mask[0]), dtype='uint8')
    Y_new = cv2.resize(Y_new, dsize=(400, 400), interpolation=cv2.INTER_CUBIC)
    Y_new = np.reshape(Y_new, (400, 400, 1))

    return Y_new

In [ ]:
def display(display_list):
  plt.figure(figsize=(15, 15))
  title = ["Input Image", "True Mask", "Predicted Mask"]
  for i in range(len(display_list)):
    plt.subplot(1, len(display_list), i+1)
    plt.title(title[i])
    plt.imshow(tf.keras.utils.array_to_img(display_list[i]))
    plt.axis("off")
  plt.show()

In [ ]:
def verify_metrics(images_conj, images_mask_conj):
    VP = 0
    FP = 0
    VN = 0
    FN = 0

    for i in range(len(images_conj)):
        img_rgb_ori = imread(images_conj[i])
        img_pred = segmentation_images(img_rgb_ori)
        img_mask = imread(images_mask_conj[i])
        img_mask = np.reshape(img_mask, (400, 400, 1))
        img_mask = (255-img_mask)
        display([img_rgb_ori, img_mask, img_pred])

        for c in range(0,10):
          for l in range(0,10):
            part_img_mask = img_mask[l*40:40+l*40,c*40:40+c*40]
            part_img_pred = img_pred[l*40:40+l*40,c*40:40+c*40]
            part_img_orig = img_rgb_ori[l*40:40+l*40,c*40:40+c*40,:]

            label_pred, n_pred = label(part_img_pred, connectivity=2, return_num=True)
            regions_pred = regionprops(label_pred)

            label_mask, n_orig = label(part_img_mask, connectivity=2, return_num=True)
            regions_mask = regionprops(label_mask)

            n_pred = 0
            for props_pred in regions_pred:
              if (props_pred.area > 25) and props_pred.area < 900):
                n_pred = n_pred +1

            if (n_orig == 1 and n_pred == 1):
                VP = VP + n_pred
            elif (n_orig == 1 and n_pred > 1):
                VP = VP + 1
                FP = FP + (n_pred - 1);
            elif (n_orig == 1 and n_pred == 0):
                FN = FN + n_orig
            elif (n_orig == 0 and n_pred != 0):
                FP = FP + n_pred
            elif (n_orig == 0 and n_pred == 0):
                VN = VN + 1

    print("VP: " + str(VP))
    print("FP: " + str(FP))
    print("VN: " + str(VN))
    print("FN: " + str(FN))

    Sensibilidade = VP/(VP+FN)
    Especificidade = VN/(VN+FP)
    Precisao = VP/(VP+FP)
    F1score = 2*((Precisao*Sensibilidade)/(Precisao+Sensibilidade))
    Acuracia = (VP+VN)/(VP+FN+FP+VN)

    print(" Accuracy: " + str(Acuracia))
    print(" Precision: " + str(Precisao))
    print(" Sensitivity: " + str(Sensibilidade))
    print(" Specifity: " + str(Especificidade))
    print(" F1score: " + str(F1score))

In [ ]:
path_test = 'TESTING SET'
images_conj_test_path = 'IMAGE'
images_conj_test_mask_path = 'MASK'

In [ ]:
images_conj_test = loadImages(path_test, images_conj_test_path)
images_conj_test_mask = loadImages(path_test, images_conj_test_mask_path)

In [ ]:
verify_metrics(images_conj_test, images_conj_test_mask)